<a href="https://colab.research.google.com/github/liadekel/analyzing-big-data/blob/master/HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework Assignment 1
### [The Art of Analyzing Big Data - The Data Scientist’s Toolbox](https://www.ise.bgu.ac.il/labs/fire/lectures.html)
#### By Dr. Michael Fire 

For this homework you will need to write code that analyzes real-world datasets. The code needs to be written in Python using the [sqlite3](https://docs.python.org/2/library/sqlite3.html) package. 

**Please note:** You need to answer only the questions that match your ID first digit.

# 1. Babies Names Dataset (35pt)

**Task 1 (for everyone):** Write a code that uses the  [**babies names dataset**](https://catalog.data.gov/dataset/baby-names-from-social-security-card-applications-data-by-state-and-district-of-#topic=developers_navigation) and creates a table named (Names) with the dataset data and the following columns: 'State', 'Gender', 'Name', 'Number', and 'Year' (5pt)
**Bonus:** Load the data using a Batch INSERT SQL Query (2pt)

In [0]:
# restart state
! rm -rf ./datasets

In [16]:
# Creating a dataset directory
!mkdir ./datasets
!mkdir ./datasets/us-baby-name
# download the dataset using wget
!wget --directory-prefix ./datasets/us-baby-name https://www.ssa.gov/oact/babynames/state/namesbystate.zip
!unzip ./datasets/us-baby-name/*.zip  -d ./datasets/us-baby-name/namesbystate
# concatenate to one file
!cat ./datasets/us-baby-name/namesbystate/*.TXT >> ./datasets/us-baby-name/namesbystate.txt

--2020-03-23 20:24:28--  https://www.ssa.gov/oact/babynames/state/namesbystate.zip
Resolving www.ssa.gov (www.ssa.gov)... 137.200.39.19, 2001:1930:e03::aaaa
Connecting to www.ssa.gov (www.ssa.gov)|137.200.39.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21974087 (21M) [application/zip]
Saving to: ‘./datasets/us-baby-name/namesbystate.zip’

namesbystate.zip    100%[===================>]  20.96M   999KB/s    in 22s     

2020-03-23 20:24:50 (993 KB/s) - ‘./datasets/us-baby-name/namesbystate.zip’ saved [21974087/21974087]

Archive:  ./datasets/us-baby-name/namesbystate.zip
  inflating: ./datasets/us-baby-name/namesbystate/AK.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/AL.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/AR.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/AZ.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/CA.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/CO.TXT  
  inflating: ./datasets/us-baby-name/n

In [0]:
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import os
%matplotlib inline
TEXT_PATH = './datasets/us-baby-name/namesbystate.txt'
DB_PATH = './datasets/us-baby-name/namebystate.sqlite'

In [18]:
conn = sqlite3.connect(DB_PATH) # connecting to the database
c = conn.cursor() # creating a cursor object
# create Names table
c.execute(
        '''CREATE TABLE IF NOT EXISTS Names
             ([State] text,
              [Gender] text,
              [Year] integer,
              [Name] text,
              [Number] integer)
        '''
)

#load data into convinient format
with open(TEXT_PATH) as f:
    names = f.readlines()
names = [tuple(name.strip().split(",")) for name in names]

# insert data to db
c.executemany(
    '''INSERT INTO Names(State, Gender, Number, Name, Year)
       values (?,?,?,?,?)
    ''', names
)
# show all row in Names
c.execute("SELECT * FROM Names").fetchall()

[('AK', 'F', 14, 'Mary', 1910),
 ('AK', 'F', 12, 'Annie', 1910),
 ('AK', 'F', 10, 'Anna', 1910),
 ('AK', 'F', 8, 'Margaret', 1910),
 ('AK', 'F', 7, 'Helen', 1910),
 ('AK', 'F', 6, 'Elsie', 1910),
 ('AK', 'F', 6, 'Lucy', 1910),
 ('AK', 'F', 5, 'Dorothy', 1910),
 ('AK', 'F', 12, 'Mary', 1911),
 ('AK', 'F', 7, 'Margaret', 1911),
 ('AK', 'F', 7, 'Ruth', 1911),
 ('AK', 'F', 6, 'Annie', 1911),
 ('AK', 'F', 6, 'Elizabeth', 1911),
 ('AK', 'F', 6, 'Helen', 1911),
 ('AK', 'F', 9, 'Mary', 1912),
 ('AK', 'F', 8, 'Elsie', 1912),
 ('AK', 'F', 7, 'Agnes', 1912),
 ('AK', 'F', 7, 'Anna', 1912),
 ('AK', 'F', 7, 'Helen', 1912),
 ('AK', 'F', 7, 'Louise', 1912),
 ('AK', 'F', 6, 'Jean', 1912),
 ('AK', 'F', 6, 'Ruth', 1912),
 ('AK', 'F', 5, 'Alice', 1912),
 ('AK', 'F', 5, 'Esther', 1912),
 ('AK', 'F', 5, 'Ethel', 1912),
 ('AK', 'F', 5, 'Margaret', 1912),
 ('AK', 'F', 5, 'Marie', 1912),
 ('AK', 'F', 21, 'Mary', 1913),
 ('AK', 'F', 9, 'Elizabeth', 1913),
 ('AK', 'F', 8, 'Margaret', 1913),
 ('AK', 'F', 7, 'Hele

**Task 2 (for everyone):** Write a query that returns the statistics for the name William (5pt). Use the [the timeit package](https://docs.python.org/3/library/timeit.html) to measure the time it takes the query to run (5pt). **Bonus:** [Create an index](https://www.w3schools.com/sql/sql_create_index.asp)  on the _Name_ column and use the [the timeit package](https://docs.python.org/3/library/timeit.html) to measure the time it takes the query to run with the index (5pt)

In [47]:
import time
import timeit

def test():
    query = "SELECT COUNT(*) FROM Names WHERE Name='William'"
    print("There are {} williams".format(c.execute(query).fetchone()[0]))

c.execute("DROP INDEX IF EXISTS idx_name")
print("Time without index:{}".format(timeit.timeit("test()", globals=globals(), number=1)))

c.execute('''CREATE INDEX IF NOT EXISTS idx_name
             ON Names (name);''')
print("Time with index:{}".format(timeit.timeit("test()", globals=globals(), number=1)))

There are 6726 williams
Time without index:0.4382013520007604
There are 6726 williams
Time with index:0.0005346359976101667


### <span style="color:red"> Please answer only **one** of the following questions according to your ID number (use the formula **<YOUR_ID> mod 4 +1**) </span>

In [37]:
# which question to answer - put your ID number and run the code 
your_id  = "316460443"
q = int(your_id) % 4 + 1
print("You need to answer question number %s" % q)

You need to answer question number 4


***Question 1:*** Write a function that returns how many babies were born in a given state in a given year.
Use it to calculate the number of babies born in LA in 1950 (10pt)

***Question 2:*** Write a function that returns how many male babies were born between a given range of years.
Use it to calculate how many babies were born between 1970 and 1975  (10pt)

**Question 3:** Write a function that returns the most common female name in a given state. Use it to calculate the most common female name in Wasington in 1987 (10pt)

**Question 4:** Write a function that returns how many male babies named _William_ where born in a given state in a given year. Use it to find the state in which the highest number of babies _William_ where born in 1999 (10pt)

**Question (for everyone):** For the state of NY write code that calculates the second most popular female/male names in each decade (10pt). **Bonus**: Visualize it somehow using Matplotlib (5pt)

# 2. Flavors of Cacao Dataset (15pt)

Using the [Flavors of Cacao](https://www.kaggle.com/rombikuboktaeder/choco-flavors) dataset, answer the following questions:

**Question 1:** Write a function that returns the number of bars manufactured where the bars' BroadBean Origin is a given country. Use the function to calculate the number of bars where BroadBean Origin is 'Fiji' (15pt)

**Question 2:** Write a function that returns the maximal and average cocoa percentage in a bar manufactured by a company in a specific country. Use the function to calculate the maximal and average cocoa percentage in bars manufactured by a Swiss company (15pt).

**Question 3:** Calculate the second most common bean type(s) and the most rare bean type(s) (15
pt)

**Question 4:** Calculate the number of reviews and the average rating in each year. Calculate the number of reviews and the average rating of each company in each year (15pt)

# 3. Kickstarter Projects Dataset (25pt)

Using the [Kickstarter Projects Dataset](https://www.kaggle.com/kemical/kickstarter-projects#ks-projects-201801.csv), answer the following questions:

**Task 1 (for everyone):** Load the dataset to SQLite DB using [PonyORM](https://ponyorm.org) (10pt)

### <span style="color:red"> Please answer only **one** of the following questions according to your ID number (use the formula **<YOUR_ID> mod 3 +1**) </span>

In [0]:
# which question to answer - put your ID number and run the code 
your_id  = "<fill_your_id>"
q = int(your_id) % 3 + 1
print("You need to answer question number %s" % q)

**Question 1:** On average which project category received the highest number of backers? (15 pt)

**Question 2:** On average which project category received the highest pledged USD? (15 pt)

**Question 3:** In which month occurred the highest number of projects? (15 pt)

## 4. Oscars Datasets (10pt)

Using the [Oscars Dataset](https://www.kaggle.com/theacademy/academy-awards), please answer only one of the following questions (you can chose):

**Question 1:** Who is the female actress with the most Oscar nominees? (10pt)

**Question 2:** Who is the male director with the most Oscar nominees? (10pt)

**Question 3:** Which top-10 movies received the highest number of Oscar nominees? (10pt)

**Question 4:** Write a function that receives an actor's name and returns the actor’s number of Oscar nominees. Use the function to calculate the number of times Leonardo DiCaprio was a nominee (10pt)

## 5. Select a Dataset (15pt)

**Open Question:** Select an interesting dataset and use SQL to discover something interesting (15pt). **Bonus:** Use BigQuery (2pt)